In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import gc

%load_ext autoreload
%autoreload 2

In [2]:
home_path = os.getenv('HOME') # создаем домашнюю директорию
dir_project = 'proj_news_radar/proj_news_viz/nlp' # прописываем путь проекта

PATH = os.path.join(home_path, dir_project)
sys.path.append(PATH)

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

---